# Bi-level Locality Sensitive Hashing for K-Nearest Neighbor Computation

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import cauchy
from sklearn.metrics import euclidean_distances
import scipy.stats as stats
import scipy.spatial.distance as dis
import math
from sklearn.preprocessing import Normalizer
import sys
from hashlib import md5
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

mbyte = 1048576

norm_l1 = Normalizer(norm='l1')
norm_l2 = Normalizer(norm='l2')
std_scaler = StandardScaler()

## tools

In [ ]:
def generate_md5(H):
    hmd5 = md5()
    hmd5.update(str(H).encode(encoding='utf-8'))
    return hmd5.hexdigest()

## Metrics

In [ ]:
# metrics
def metrics(candidates, approximate_neighbors, exact_neighbors, q, P):
    # recall
    recall = recall_ratio(candidates, exact_neighbors)
    # error
    error = error_ratio(q, exact_neighbors, approximate_neighbors, P) 
    # selectivity
    s = selectivity(candidates, P)
    return recall, error, s

# recall ratio
def recall_ratio(candidates, exact_neighbors):
    return len(np.intersect1d(candidates, exact_neighbors)) / len(exact_neighbors)

# error ratio
def error_ratio(v, exact_neighbors, approximate_neighbors, P):
    k = len(approximate_neighbors)
    exact = np.array([dis.euclidean(v, P[idx]) for idx in exact_neighbors])[:k]
    approximate = np.array([dis.euclidean(v, P[idx]) for idx in approximate_neighbors])
    return np.sum(exact / approximate) / k

# selectivety
def selectivity(candidates, P):
    return len(candidates) / len(P)

In [ ]:
metric_results = dict()

## datasets

In [ ]:
# 最近邻数据
%store -r Datasets

In [ ]:
Datasets.keys()

In [ ]:
data = Datasets['SIFT-10K']

train_sift = data['train']
test_sift = data['test']

k_near_neighbors = data['k_near_neighbors']

## Random Projection

In [ ]:
P = datasets.make_blobs(centers=1, n_features=2, n_samples=10000, random_state=1, cluster_std=5)[0]

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(P[:, 0], P[:, 1], marker='x')
plt.show()

### Compute Diameter

In [ ]:
def compute_diamter(P, n=40):
    idxs = np.arange(len(P))
    
    idx = np.random.randint(len(P))
    p = P[idx]
    q1_idx = find_farthest(P, p)
    q1 = P[q1_idx]
    q2_idx = find_farthest(P, q1)
    q2 = P[q2_idx]
    rho = dis.euclidean(p, q1)
    r = dis.euclidean(q1, q2)
    r_arr = [r]
    
    for i in range(1, n):
        if p in P:
            np.delete(P, idx)
        if q1 in P:
            np.delete(P, q1_idx)
        if len(P) <= 1:
            break

        p1 = q1 + (r / rho) * (p - q1)
        p = 0.5 * (p1 + q1)
        q1_idx = find_farthest(P, p)
        q1 = P[q1_idx]
        q2_idx = find_farthest(P, q1)
        q2 = P[q2_idx]
        rho = dis.euclidean(p, q1)
        r = dis.euclidean(q1, q2)

        if r <= r_arr[i - 1]:
            return r_arr

        r_arr.append(r)

    return r_arr

In [ ]:
def find_farthest(P, q):
    return np.argsort(np.array([dis.euclidean(p, q) for p in P]))[-1]

In [ ]:
%%time
euclidean_distances(P).max()

In [ ]:
%%time
compute_diamter(P)

In [ ]:
idxs = np.arange(len(P))
v = P[np.random.choice(idxs, 2, replace=False)]
v = v[1]- v[0]

projection = P.dot(v)
mid = np.median(projection)

idxs_R = np.argwhere(projection >= mid)
idxs_L = np.argwhere(projection < mid)
R = idxs[np.concatenate(idxs_R)]
L = idxs[np.concatenate(idxs_L)]

In [ ]:
plt.plot([0, v[0]], [0, v[1]])
plt.scatter(P[L, 0], P[L, 1])
plt.scatter(P[R, 0], P[R, 1])
plt.show()

### Normal

In [ ]:
def rp_forest_normal(P, n_trees=10, max_depth=None, min_size=None, random_state=None):
    """
    Desc:
        Only support the leafs arg.
    
    Args:
        n_trees: number of trees
        max_depth: default(None)
        random_state
    
    Returns:
        forest
        
    """
    assert max_depth is not None or min_size is not None, \
        "Should give one of the value of max_depth and min_size"

    if random_state is None:
        rand = np.random.RandomState()
    else:
        rand = np.random.RandomState(random_state)
    
    forest = []
    n, d = P.shape
    s_idxs = [] # stack of current index of points
                # idxs, parent_node, direction, depth

    for i in range(n_trees): # build forest
        tree = dict()
        s_idxs.append((np.arange(n), tree, 'root', 0, '')) # root node

        while len(s_idxs) != 0:
            idxs, parent_node, direc, depth, code = s_idxs.pop()

            node = dict()

            if depth >= max_depth:
                node['idxs'] = idxs
                parent_node[direc] = node
                node['code'] = int(code, 2)
                continue

            v = idxs[rand.choice(np.arange(len(idxs)), 2, replace=False)] # choose two points randomly
            v = P[v[1]]- P[v[0]]
            
            projection = P[idxs].dot(v)
            mid = np.median(projection)
            
            node['v'] = v
            node['split'] = mid

            idxs_R = np.argwhere(projection >= mid)
            idxs_L = np.argwhere(projection < mid)

            if len(idxs_R) == 0 or len(idxs_L) == 0: # 划分失败
                continue
            
            parent_node[direc] = node
            
            R = idxs[np.concatenate(idxs_R)]
            L = idxs[np.concatenate(idxs_L)]

            s_idxs.append((R, node, 'r', depth + 1, code + '0'))
            s_idxs.append((L, node, 'l', depth + 1, code + '1'))
            
        forest.append(tree)
        
    return forest

#### Build Tree

In [ ]:
%%time
forest = rp_forest_normal(P, max_depth=8, random_state=0)

In [ ]:
print('random projection tree (mean)')
tree = forest[0]
plt.figure(figsize=(15, 10))
stack = [tree['root']]
while len(stack) != 0:
    node = stack.pop()
    if 'idxs' not in node:
        stack.append(node['r'])
        stack.append(node['l'])
    else:
        plt.scatter(P[node['idxs'], 0], P[node['idxs'], 1], marker='x', alpha=1)

plt.show()

#### Search

In [ ]:
def search_tree_normal(tree, q):
    node = tree['root']
    while 'idxs' not in node:
        projection = q.dot(node['v'])
        if projection >= node['split']:
            node = node['r']
        else:
            node = node['l']
    return node['code']

In [ ]:
def search_leaf_normal(tree, q):
    node = tree['root']
    while 'idxs' not in node:
        projection = q.dot(node['v'])
        if projection >= node['split']:
            node = node['r']
        else:
            node = node['l']
    return node['idxs']

In [ ]:
search_tree_normal(forest[0], P[0])

In [ ]:
def find_nearest_neighbors_normal(forest, q):
    candidates = []
    for tree in forest:
        candidates.append(search_tree_normal(tree, q))
    if len(candidates) != 0:
        candidates = np.unique(np.concatenate(np.array(candidates)))
    
    return candidates

### Mean

In [ ]:
def rp_forest_mean(P, n_trees=10, c=4, max_depth=None, min_size=None, random_state=None):
    """
    Desc:
        Now only support the leafs arg.
    
    Args:
        n_trees: number of trees
        depth:
        random_state
    
    Returns:
        split
        forest
        forest_leafs
        
    """
    assert max_depth is not None or min_size is not None, \
        "Should give one of the value of max_depth or min_size"

    if random_state is None:
        rand = np.random.RandomState()
    else:
        rand = np.random.RandomState(random_state)

    forest = []
    n, d = P.shape
    V = dict()
    s_idxs = []

    for i in range(n_trees):
        tree = dict()
        s_idxs.append((np.arange(n), tree, 'root', 0, ''))

        while len(s_idxs) != 0:
            idxs, parent_node, direc, depth, code = s_idxs.pop()

            node = dict()

            if depth >= max_depth:
                node['idxs'] = idxs
                parent_node[direc] = node
                node['code'] = int(code, 2)
                continue
                
            v = idxs[rand.choice(np.arange(len(idxs)), 2, replace=False)]
            v = P[v[1]]- P[v[0]]
            node['v'] = v
            
            diameter = compute_diamter(P[idxs])[-1]

            mean_P = P[idxs].mean(axis=0)
            dis_P = np.array([dis.euclidean(mean_P, p) for p in P[idxs]])
            avg_2_diameter = 2 * np.sum(dis_P ** 2) / len(idxs)
#             diameters = []
#             select_idxs = idxs[rand.choice(np.arange(len(idxs)), 40, replace=False)]
#             for idx in select_idxs:
#                 diameters.append(np.array([dis.euclidean(P[idx], p) for p in P[idxs]]).max())
#             avg_diameters = np.array(diameters).mean()
#             avg_2_diameters = avg_diameters ** 2
            
            diameter_2 = diameter ** 2
            # print(diameter_2, avg_2_diameter)

            node['mean'] = mean_P
            
            if diameter_2 <= c * avg_2_diameter:
                projection = P[idxs].dot(v)
                split = np.median(projection)
                node['split'] = split
                node['type'] = 0
            else:
                projection = dis_P
                split = np.median(projection)
                node['split'] = split
                node['type'] = 1

            idxs_R = np.argwhere(projection > split)
            idxs_L = np.argwhere(projection <= split)

            # print(len(idxs_R), len(idxs_L))
            
            if len(idxs_R) != 0:
                R = idxs[np.concatenate(idxs_R)]
            if len(idxs_L) != 0:
                L = idxs[np.concatenate(idxs_L)]
            
            parent_node[direc] = node
            
            # print(direc)
            s_idxs.append((R, node, 'r', depth + 1, code + '0'))
            s_idxs.append((L, node, 'l', depth + 1, code + '1'))
            
        forest.append(tree)

    return forest

#### Build Tree

In [ ]:
%%time
forest = rp_forest_mean(P, c=4, max_depth=4, random_state=1)

In [ ]:
print('random projection tree (mean)')
tree = forest[0]
plt.figure(figsize=(15, 10))
stack = [tree['root']]
while len(stack) != 0:
    node = stack.pop()
    if 'idxs' not in node:
        stack.append(node['r'])
        stack.append(node['l'])
    else:
        plt.scatter(P[node['idxs'], 0], P[node['idxs'], 1], marker='x', alpha=1)

plt.show()

In [ ]:
def search_tree_mean(tree, q):
    node = tree['root']
    while 'idxs' not in node:
        if node['type'] == 1:
            projection = np.sqrt(np.sum((q - node['mean']) ** 2)) 
        else:
            projection = q.dot(node['v'])

        if projection > node['split']:
            node = node['r']
        else:
            node = node['l']
    return node['idxs']

In [ ]:
def find_nearest_neighbors_mean(forest, q):
    candidates = []
    for tree in forest:
        candidates.append(search_tree_mean(tree, q))
    if len(candidates) != 0:
        candidates = np.unique(np.concatenate(np.array(candidates)))
    
    return candidates

#### Search

In [ ]:
%%time
q_idx = np.random.randint(10000)
print('find P[{}] nearest neighbors'.format(q_idx))
q = P[q_idx]
# candidates = search_tree(forest[0], q)
candidates = find_nearest_neighbors_mean(forest, q)
print('number of candidates: {}'.format(len(candidates)))
plt.figure(figsize=(15, 10))
plt.scatter(P[:, 0], P[:, 1])
plt.scatter(P[candidates, 0], P[candidates, 1])
plt.scatter(q[0], q[1], c='r')
plt.show()

### Max

#### Build Tree

In [ ]:
def rp_forest_max(P, n_trees=10, max_depth=None, min_size=None, random_state=None):
    """
    Desc:
        Now only support the leafs arg.
    
    Args:
        n_trees: number of trees
        depth:
        random_state
    
    Returns:
        split
        forest
        forest_leafs
        
    """
    assert max_depth is not None or min_size is not None, \
        "Should give one of the value of max_depth or min_size"

    if random_state is None:
        rand = np.random.RandomState()
    else:
        rand = np.random.RandomState(random_state)
    
    
    forest = []
    n, d = P.shape
    s_idxs = []

    for i in range(n_trees):
        tree = dict()
        s_idxs.append((np.arange(n), tree, 'root', 0, ''))

        while len(s_idxs) != 0:
            idxs, parent_node, direc, depth, code = s_idxs.pop()

            node = dict()

            if depth >= max_depth:
                node['idxs'] = idxs
                parent_node[direc] = node
                node['code'] = int(code, 2)
                continue

            v = idxs[rand.choice(np.arange(len(idxs)), 2, replace=False)]
            v = P[v[1]]- P[v[0]]
            node['v'] = v

            x = P[idxs[rand.randint(len(idxs))]]
            diameter = np.array([dis.euclidean(x, p) for p in P[idxs]]).max()
            delta = (rand.rand() * 2 - 1) * diameter / np.sqrt(d)
            
            projection = P[idxs].dot(v)

            split = np.median(projection) + delta

            node['split'] = split            

            idxs_R = np.argwhere(projection > split)
            idxs_L = np.argwhere(projection <= split)

            if len(idxs_R) != 0:
                R = idxs[np.concatenate(idxs_R)]
            if len(idxs_L) != 0:
                L = idxs[np.concatenate(idxs_L)]
            
            parent_node[direc] = node

            s_idxs.append((R, node, 'r', depth + 1, code + '0'))
            s_idxs.append((L, node, 'l', depth + 1, code + '1'))
            
        forest.append(tree)
        
    return forest

In [ ]:
%%time
forest = rp_forest_max(P, max_depth=4, random_state=1)

In [ ]:
print('random projection tree (mean)')
tree = forest[0]
plt.figure(figsize=(15, 10))
stack = [tree['root']]
while len(stack) != 0:
    node = stack.pop()
    if 'idxs' not in node:
        stack.append(node['r'])
        stack.append(node['l'])
    else:
        plt.scatter(P[node['idxs'], 0], P[node['idxs'], 1], marker='x', alpha=1)

plt.show()

In [ ]:
def search_tree_max(tree, q):
    node = tree['root']
    while 'idxs' not in node:
        projection = q.dot(node['v'])
        if projection > node['split']:
            node = node['r']
        else:
            node = node['l']
            
    return node['idxs']

In [ ]:
def find_nearest_neighbors_max(forest, q):
    candidates = []
    for tree in forest:
        candidates.append(search_tree_max(tree, q))
    if len(candidates) != 0:
        candidates = np.unique(np.concatenate(np.array(candidates)))
    
    return candidates

#### Search

In [ ]:
%%time
q_idx = np.random.randint(10000)
print('find P[{}] nearest neighbors'.format(q_idx))
q = P[q_idx]
# candidates = search_tree(forest[0], q)
candidates = find_nearest_neighbors_max(forest, q)
print('number of candidates: {}'.format(len(candidates)))
plt.figure(figsize=(15, 10))
plt.scatter(P[:, 0], P[:, 1], marker='x')
plt.scatter(P[candidates, 0], P[candidates, 1], marker='x')
plt.scatter(q[0], q[1], c='r')
plt.show()

In [ ]:
class BiLevelIndex(object):
    def __init__(self, dataset):
        self.dataset_ = dataset
        self.n_, self.d_ = dataset.shape
        
    def build_forest(self, n_trees=10):
        pass
    
    def get_nns(self, query, k=1):
        pass
    
    def recall(self, test):
        pass
    
    def precision(self, test):
        pass
    
    def hashing_time(self, test):
        pass
    
    def query_time(self, test):
        pass
    
    def score(self, test):
        pass

### RP tree 3rd

In [ ]:
def rp_forest_3rd(P, n_trees=10, max_depth=None, min_size=None, random_state=None):
    """
    Desc:
        Now only support the leafs arg.
    
    Args:
        n_trees: number of trees
        depth:
        random_state
    
    Returns:
        split
        forest
        forest_leafs
        
    """
    assert max_depth is not None or min_size is not None, \
        "Should give one of the value of max_depth or min_size"

    if random_state is None:
        rand = np.random.RandomState()
    else:
        rand = np.random.RandomState(random_state)
    
    c = 2
    forest = []
    n, d = P.shape
    s_idxs = []

    for i in range(n_trees):
        tree = dict()
        s_idxs.append((np.arange(n), tree, 'root', 0))

        while len(s_idxs) != 0:
            idxs, parent_node, direc, depth = s_idxs.pop()

            node = dict()

            if depth >= max_depth:
                node['idxs'] = idxs
                parent_node[direc] = node
                continue

            v = idxs[rand.choice(np.arange(len(idxs)), 2, replace=False)]
            v = P[v[1]] - P[v[0]]
            node['v'] = v

            diameter = compute_diamter(P[idxs])

            diameters = []
            select_idxs = idxs[rand.choice(np.arange(len(idxs)), 40, replace=False)]
            for idx in select_idxs:
                diameters.append(np.array([dis.euclidean(P[idx], p) for p in P[idxs]]).max())
            avg_diamters = diameters.mean()
            avg_2_diamters = avg_diamters ** 2
            
            diameter_2 = diameter ** 2
            print(diameter_2, avg_2_diamters)
            
            if diameter_2 <= c * avg_2_diamters:
                projection = P[idxs].dot(v)
                
            split = np.median(projection) + delta
            projection = P[idxs].dot(v)
            node['split'] = split            

            # print(projection)
            idxs_R = np.argwhere(projection > split)
            idxs_L = np.argwhere(projection <= split)
        
            # print(len(idxs_R), len(idxs_L))
            
            if len(idxs_R) != 0:
                R = idxs[np.concatenate(idxs_R)]
            if len(idxs_L) != 0:
                L = idxs[np.concatenate(idxs_L)]
            
            parent_node[direc] = node
            
            # print(direc)
            s_idxs.append((R, node, 'r', depth + 1))
            s_idxs.append((L, node, 'l', depth + 1))
            
        forest.append(tree)
        
    return forest

## Modeling LSH for Performan Tuning

$$
f_{\kappa, \theta}(x) = \alpha(\frac{x}{\theta})^\kappa e^{-x/\theta}
$$

$$
D = 2(\kappa + 1)
$$

E: 算术平均值  

G: 几何平均值

$$
\begin{cases}
\kappa \theta = E \\
\text{ln}(\kappa) - \psi(\kappa) = \text{ln}(E) - \text{ln}(G)   
\end{cases} \\
\psi(x) = \frac{\Gamma^{'}(x)}{\Gamma(x)}
$$

### For $X^2$

In [ ]:
def gmean(arr):
    return arr.prod() ** (1.0/len(arr))

In [ ]:
from scipy.spatial.distance import euclidean
from scipy.stats.mstats import gmean
from scipy.special import digamma
from scipy.stats import gamma

In [ ]:
n_select = 1000
idxs_select = np.random.choice(np.arange(10000), (n_select, 2), replace=False)
d_select = np.array([euclidean(P[idxs[0]], P[idxs[1]]) for idxs in idxs_select])

In [ ]:
E = (d_select**2).mean()
G = gmean(d_select**2)
print('E: {:.4f}, G: {:.4f}'.format(E, G))

In [ ]:
equation_right = np.log(E) - np.log(G)
Kappa = np.arange(1, 5, 0.5)
result = np.log(Kappa) - digamma(Kappa)
plt.plot(Kappa, result)
plt.xticks(Kappa)
plt.hlines(y=equation_right, xmin=Kappa[0], xmax=Kappa[-1])
plt.xlabel('kappa')
plt.ylabel('equation left')
plt.show()

In [ ]:
kappa = 1
theta = E / kappa
print('kappa: {}, theta: {:.4f}'.format(kappa, theta))

In [ ]:
ds = np.array([[euclidean(P[i], p) for p in P] for i in np.random.choice(np.arange(10000), 40)])

In [ ]:
ds = np.sort(ds)

In [ ]:
order_ds = ds.mean(axis=0)

In [ ]:
pp = np.linspace(gamma.ppf(0.01, a=kappa, scale=theta), gamma.ppf(0.99, a=kappa, scale=theta), 100)
pd = gamma.pdf(pp, a=kappa, scale=theta)

In [ ]:
plt.hist(order_ds**2, bins=100, density=True)
plt.plot(pp, pd)
plt.show()

### For $X_k^2$

$$
E[X_k] = \frac{[\Gamma(1+\frac{D_1}{2})]^{\frac{1}{D_1}}}{\sqrt \pi}(\frac{k}{N-1})^\frac{1}{D_2}
$$

$$
E_k = \alpha k^\beta N^\gamma \\
G_k = \alpha ^{'} k^{\beta'} N^{\gamma'}
$$

In [ ]:
n_anchors = 100
n = 10000
idxs_P = np.arange(n)
idxs_anchor = np.random.choice(np.arange(10000), n_anchors, replace=False)

idxs_rest = np.setdiff1d(idxs_P, idxs_anchor)

points_anchor = P[idxs_anchor]

In [ ]:
N = np.arange(1000, len(idxs_rest), 1000)
K = np.arange(50, 501, 50)

In [ ]:
idxs_samples = np.array([np.random.choice(np.arange(len(idxs_rest)), n, replace=False) for n in N])

In [ ]:
%%time
d_knns = []
for q in points_anchor:
    d_ns = []
    for idxs_sample in idxs_samples:
        d_ks = []
        ds = np.array([euclidean(q, p) for p in P[idxs_sample]])
        sort_ds = np.sort(ds)
        for k in K:
            d_ks.append(sort_ds[k])
        d_ns.append(d_ks)
    d_knns.append(d_ns)

In [ ]:
d_knns = np.array(d_knns)

In [ ]:
d_knns.shape

In [ ]:
d_1e4_knns = d_knns[:, -1, :]

In [ ]:
fix_n_1e4_mean = (d_1e4_knns ** 2).mean(axis=0)

In [ ]:
fix_n_1e4_gmean = (d_1e4_knns ** 2).prod(axis=0) ** (1 / n_anchors)

In [ ]:
plt.plot(K, fix_n_1e4_mean, label='mean', marker='x')
plt.plot(K, fix_n_1e4_gmean, label='gmean', marker='>')
plt.legend()
plt.show()

In [ ]:
nks = d_knns.mean(axis=0)

In [ ]:
nks

## Standard LSH

In [ ]:
def compute_pr(w, c):
    """
    Args:
        w: 段长
        c: r1, r2, 距离
    """
    a = 2 * norm.cdf(-w/c)
    b = 2 / (np.sqrt(2 * np.pi) * w / c)
    d = np.e ** (-((w**2) / (2 * (c ** 2))))    
    return 1 - a - b * (1 - d)
    
def compute_rho(p1, p2):
    return (np.log(1/p1) / np.log(1/p2))

def compute_r1(P):
    d = P.shape[1]
    print('Estimate r1')
    print('P: {}'.format(P.shape))

#     n = P.shape[0] * 0.03
    n = 20
    nn = int(np.log(P.shape[0]))
    print('sample {} points, calculate near {} neighbors.'.format(n, nn))
    idx = np.random.choice(P.shape[0], size=n, replace=False)
    qs = P[idx]
    r1_avg = []
    for q in qs:
        dist_arr = np.array([dis.euclidean(q, p) for p in P])
        dist_sort = np.sort(dist_arr)
        avg = np.sort(dist_arr)[1:nn+1].mean()
        r1_avg.append(avg)

    r1 = np.array(r1_avg).mean()
    print('estimate r1 = {}'.format(r1))
    return r1

def draw_W(P, c=2):
    """
    Desc:
        绘制图像，观察最优 w 值
    Args:
        c: 近似因子
    """
    r1 = compute_r1(P)
    r2 = c * r1
    W = np.arange(r1 / 2, r1 * 10, r1 / 2)

    p1 = [compute_pr(w, r1) for w in W]
    p2 = [compute_pr(w, c * r1) for w in W]
    rho = [compute_rho(p1[i], p2[i]) for i in range(len(W))]

    plt.plot(W, p1, label='p1')
    plt.plot(W, p2, label='p2')
    plt.plot(W, rho, label='rho')
    plt.hlines(y=1/c, xmin=0, xmax=r1 * 10, label='1/c')
    plt.xlabel('w')
    plt.ylabel('p')
    plt.legend()
    plt.show()

In [ ]:
def generate_AB(d, M, W, L, random_state=None):
    """
    Args:
        d: dim
        M: same as k
        W: interval
        L: number of hash tables
    Returns:
        A: L * M * d
        B: L * M
    """
    if random_state is None:
        rand = np.random.RandomState()
    else:
        rand = np.random.RandomState(random_state)
    A = rand.normal(0, 1, (L, M, d))
    B = rand.uniform(0, W, (L, M))
    return A, B

In [ ]:
def hashing(v, a, b, W):
    return (a.dot(v) + b) / W

## $Z^8$

## Morton Curve

In [ ]:
V = np.random.randint(0, 6, (10000, 8))

In [ ]:
max_bit = len(bin(V.max())[2:])
max_bit

In [ ]:
def morton_curve(v, max_bit):
    bin_codes = np.array([bin(i) for i in v])
    codes = []
    for i, code in enumerate(bin_codes):
        codes.append(code[2:])
        n = len(codes[i])
        for j in range(max_bit - n):
            codes[i] = '0' + codes[i]

    result = ''.join(np.array(list(map(list, codes))).T.reshape(1, -1)[0])
    return int(result, 2)

In [ ]:
%%time
morton_numbers = [morton_curve(v, max_bit) for v in V]
morton_numbers.sort()

**四的倍数？如何判断相邻？**

## $E_8$ Lattice

In [ ]:
v = np.random.rand(8) + np.random.randint(0, 10, 8)

In [ ]:
v

In [ ]:
v1 = v.round()
v1

In [ ]:
v2 = (v - 1/2).round() + 1/2
v2

In [ ]:
dis.euclidean(v, v1), dis.euclidean(v, v2)

In [ ]:
# D8

## Multi Probe

## Experiment

### $Z^M$

In [ ]:
L = [10, 20, 30]
M = 8
k = 50
n, d = train_sift.shape

#### Level 1

In [ ]:
%%time
forest = rp_forest_normal(P=train_sift, max_depth=4)

In [ ]:
print('random projection tree (mean)')
tree = forest[0]
stack = [(tree['root'], '')]

while len(stack) != 0:
    node, code = stack.pop()
    if 'idxs' not in node:
        stack.append((node['r'], code + '0'))
        stack.append((node['l'], code + '1'))
    else:
        print(int(code, 2))
        print(len(node['idxs']))
        draw_W(train_sift[node['idxs']])
        idxs = node['idxs']

#### Level 2

generate A and B

In [ ]:
draw_W(train_sift)

In [ ]:
W = np.arange(500, 2100, 100)
W

In [ ]:
A, B = generate_AB(d=d, L=L[0], M=M, W=W[-1])

In [ ]:
print('Level 2: Store points into buckets')

def construct_tables(forest, P, A, B, L, W):
    trees = [] # n trees
    for tree in forest:
        nodes = dict() # 16 nodes
        stack = [tree['root']]
        while len(stack) != 0:
            node = stack.pop()
            if 'idxs' not in node:
                stack.append((node['r']))
                stack.append((node['l']))
            else:
                code = node['code']
                tables = [] # L buckets
                idxs = node['idxs']

                # Level 2
                for i in range(L):
                    buckets = dict() # n bucket
                    H = np.floor((P[idxs].dot(A[i].T) + B[i]) / W)
                    for j, h in enumerate(H):
                        bi = generate_md5(h)
                        if bi in buckets:
                            buckets[bi].append(idxs[j])
                        else:
                            buckets[bi] = [idxs[j]]
                    tables.append(buckets)

                nodes[code] = tables

        trees.append(nodes)

    return trees

- trees: array, n 棵树
    - nodes: dict, 16 个叶子节点, key=code
        - tables: array, L 个哈希表
            - buckets: dict, m 个哈希桶, key=bucket_index

In [ ]:
%%time
trees = construct_tables(forest, train_sift, A, B, L[0], W[-1])

#### Query

In [ ]:
q = test_sift[0]

#### Find Tree Node Code

In [ ]:
def find_leaf_codes(forest, q):
    leaf_codes = []
    for tree in forest:
        leaf_codes.append(search_tree_normal(tree, q))
    return leaf_codes

In [ ]:
leaf_codes = find_leaf_codes(forest, q)
leaf_codes

#### Candidates

In [ ]:
def find_near_neighbors(trees, leaf_codes, q, A, B, L, W):
    result = []
    for i, code in enumerate(leaf_codes):
        tree = trees[i]
        node = tree[code]
        for j in range(L):
            h = np.floor((A[j].dot(q) + B[j]) / W)
            bi = generate_md5(h)
            if bi in node[j]:
                result.append(node[j][bi])
    return np.unique(np.concatenate(np.array(result)))

In [ ]:
candidates = find_near_neighbors(trees, leaf_codes, q, A, B, L[0], W[-1]) 

#### Find k Nearest Neighbors

In [ ]:
def find_k_nearest_neighbors(P, q, near_neighbors_idxs, K=50):
    if len(near_neighbors_idxs) < K:
        print('number of candidates less than K')
        print('K = number of candidates')
        K = len(near_neighbors_idxs)
    return near_neighbors_idxs[np.argsort(np.array(
        [dis.euclidean(q, p) for p in P[near_neighbors_idxs]])
                                         )[:K]
                              ]

In [ ]:
approximate_neighbors = find_k_nearest_neighbors(train_sift, q, result)

#### Metrics

In [ ]:
recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[0], 
                           q, train_sift)

print('recall = {:.4f}'.format(recall))
print('error  = {:.4f}'.format(error))
print('selevtivity = {:.4f}'.format(s))

#### Integration

In [ ]:
L = [10]
W =  np.arange(1500, 2100, 100)
M = 8
K = 50
n, d = train_sift.shape

In [ ]:
%%time

results = []

for l in L:
    for w in W:
        # 1. 构造 rp-tree
        forest = rp_forest_normal(P=train_sift, max_depth=4)
        # 2. 构造 hash table
        A, B = generate_AB(d=d, L=l, M=M, W=w)
        trees = construct_tables(forest, train_sift, A, B, l, w)
        
        result = dict()
        result['params'] = dict()
        result['params']['L'] = l
        result['params']['W'] = w
        
        print('L = {}, W = {}'.format(l, w))

        recall_arr = []
        error_arr = []
        selectivity_arr = []
        # 3. 查询点
        for i, q in enumerate(test_sift):
            leaf_codes = find_leaf_codes(forest, q)
            
            # 4. 获取 候选集 candidates
            candidates = find_near_neighbors(trees, leaf_codes, q, A, B, l, w) 

            # 5. 计算 近似 k近邻
            approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

            # 6. 评估
            recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[i], 
                           q, train_sift)
            
            recall_arr.append(recall)
            error_arr.append(error)
            selectivity_arr.append(s)
        result['metrics'] = dict()
        result['metrics']['recall'] = sum(recall_arr) / len(test_sift)
        result['metrics']['error'] = sum(error_arr) / len(test_sift)
        result['metrics']['selectivity'] = sum(selectivity_arr) / len(test_sift)
    
        metric_results['ZM'].append(result)
#         results.append(result)
    
# metric_results['ZM'] = results

In [ ]:
metric_results['ZM']

In [ ]:
l = 10
w = 800
# 1. 构造 rp 树
forest = rp_forest_normal(P=train_sift, max_depth=4)
# 2. 构造 hash table
A, B = generate_AB(d=d, L=l, M=M, W=w)
trees = construct_tables(forest, train_sift, A, B, l, w)

recall_arr = []
error_arr = []
selectivity_arr = []
# 3. 查询点
for i, q in enumerate(test_sift):
    leaf_codes = find_leaf_codes(forest, q)

    # 4. 获取 候选集 candidates
    candidates = find_near_neighbors(trees, leaf_codes, q, A, B, l, w) 

    # 5. 计算 近似 k近邻
    approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

    # 6. 评估
    recall, error, s = metrics(candidates, 
                   approximate_neighbors, 
                   k_near_neighbors[i], 
                   q, train_sift)

    recall_arr.append(recall)
    error_arr.append(error)
    selectivity_arr.append(s)

print('L = {}, W = {}'.format(l, w))
print('recall = {:.4f}'.format(sum(recall_arr) / len(test_sift)))
print('error  = {:.4f}'.format(sum(error_arr) / len(test_sift)))
print('selevtivity = {:.4f}'.format(sum(selectivity_arr) / len(test_sift)))

In [ ]:
recall_arr = []
error_arr = []
selectivity_arr = []
for result in metric_results['ZM']:
    selectivity_arr.append(result['metrics']['selectivity'])
    recall_arr.append(result['metrics']['recall'])
    error_arr.append(result['metrics']['error'])

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='recall ratio')
plt.plot(selectivity_arr, error_arr, marker='x', label='error ratio')
plt.title('Standard, $Z^M$, M=8, L=10', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

### $E_8$

In [ ]:
def E8_decode(h, decode_type='E8'):
    D8 = h.round().astype(np.int)
    total = D8.sum()
    if total % 2 != 0:
        h2 = np.abs(np.modf(h)[0])
        h3 = h2.copy()
        # 排序
        h3[h3 < 0.5] = 0.5 - h3[h3 < 0.5]
        h3[h3 > 0.5] = h3[h3 > 0.5] - 0.5
        order = h3.argsort()

    i = 0
    while total % 2 != 0: # 或许没有必要？
        if h2[order[i]] > 0.5:
            D8[order[i]] -= 1
        else:
            D8[order[i]] += 1
        total = D8.sum()
        i += 1
    
    if decode_type == 'E8':
        D812 = E8_decode(h - 1/2, 'D8') + 1/2 # 递归求解 D8 + 1/2
        
        d1 = dis.euclidean(h, D8)
        d2 = dis.euclidean(h, D812)
        
        if d1 > d2:
            return D812
        else:
            return D8
    else:
        return D8

In [ ]:
E8 = E8_decode(h)
E8

In [ ]:
print('Level 2: Store points into buckets')

def construct_tables_E8(forest, P, A, B, L, W):
    trees = [] # n trees
    for tree in forest:
        nodes = dict() # 16 nodes
        stack = [tree['root']]
        while len(stack) != 0:
            node = stack.pop()
            if 'idxs' not in node:
                stack.append((node['r']))
                stack.append((node['l']))
            else:
                code = node['code']
                tables = [] # L buckets
                idxs = node['idxs']

                # Level 2
                for i in range(L):
                    E8 = dict() # n bucket
                    H = (P[idxs].dot(A[i].T) + B[i]) / W
                    
                    for j, h in enumerate(H):
                        E8_code = E8_decode(h)
                        bi = generate_md5(E8_code)
                        if bi in E8:
                            E8[bi].append(idxs[j])
                        else:
                            E8[bi] = [idxs[j]]
                    
                    tables.append(E8)

                nodes[code] = tables

        trees.append(nodes)

    return trees

In [ ]:
def find_near_neighbors_E8(trees, leaf_codes, q, A, B, L, W, M):
    result = []
    for i, code in enumerate(leaf_codes):
        tree = trees[i]
        node = tree[code]
        for j in range(L):
            E8 = node[j] # morton curve
            h = (A[j].dot(q) + B[j]) / W
            E8_code = E8_decode(h)
            
            bi = generate_md5(E8_code)
            if bi in E8:
                result.append(E8[bi])
                
    return np.unique(np.concatenate(np.array(result)))

#### Integration

In [ ]:
L = [10]
W =  np.arange(1400, 2100, 100)
M = 8
K = 50
n, d = train_sift.shape

In [ ]:
metric_results['E8'] = []

In [ ]:
%%time

for l in L:
    for w in W:
        # 1. 构造 rp 树
        forest = rp_forest_normal(P=train_sift, max_depth=4)
        # 2. 构造 hash table
        A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

        trees = construct_tables_E8(forest, train_sift, A, B, l, w)
        
        result = dict()
        result['params'] = dict()
        result['params']['L'] = l
        result['params']['W'] = w
        
        print('L = {}, W = {}'.format(l, w))
        recall_arr = []
        error_arr = []
        selectivity_arr = []

        # 3. 查询点
        for i, q in enumerate(test_sift):
            leaf_codes = find_leaf_codes(forest, q)

            # 4. 获取 候选集 candidates multiprobe
            candidates = find_near_neighbors_E8(trees, leaf_codes, q, A, B, l, w, M) 

            # 5. 计算 近似 k近邻
            approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)
            
            # 6. 评估
            recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[i], 
                           q, train_sift)
            
            recall_arr.append(recall)
            error_arr.append(error)
            selectivity_arr.append(s)
            
        result['metrics'] = dict()
        result['metrics']['recall'] = sum(recall_arr) / len(test_sift)
        result['metrics']['error'] = sum(error_arr) / len(test_sift)
        result['metrics']['selectivity'] = sum(selectivity_arr) / len(test_sift)
    
        metric_results['E8'].append(result)

In [ ]:
metric_results['E8']

In [ ]:
recall_arr_E8 = []
error_arr_E8 = []
selectivity_arr_E8 = []
for result in metric_results['E8']:
    selectivity_arr_E8.append(result['metrics']['selectivity'])
    recall_arr_E8.append(result['metrics']['recall'])
    error_arr_E8.append(result['metrics']['error'])

In [ ]:
plt.plot(selectivity_arr_E8, recall_arr_E8, marker='x', label='recall ratio')
plt.plot(selectivity_arr_E8, error_arr_E8, marker='x', label='error ratio')
plt.title('Standard, $E_8$, M=8, L=10', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
l = 10
w = 1100
# 1. 构造 rp 树
forest = rp_forest_normal(P=train_sift, max_depth=4)
# 2. 构造 hash table
A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

trees = construct_tables_E8(forest, train_sift, A, B, l, w)

recall_arr = []
error_arr = []
selectivity_arr = []

# 3. 查询点
for i, q in enumerate(test_sift):
    leaf_codes = find_leaf_codes(forest, q)

    # 4. 获取 候选集 candidates multiprobe
    candidates = find_near_neighbors_E8(trees, leaf_codes, q, A, B, l, w, M) 

    # 5. 计算 近似 k近邻
    approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

    # 6. 评估
    recall, error, s = metrics(candidates, 
                   approximate_neighbors, 
                   k_near_neighbors[i], 
                   q, train_sift)

    recall_arr.append(recall)
    error_arr.append(error)
    selectivity_arr.append(s)

print('L = {}, W = {}'.format(l, w))
print('recall = {:.4f}'.format(sum(recall_arr) / len(test_sift)))
print('error  = {:.4f}'.format(sum(error_arr) / len(test_sift)))
print('selevtivity = {:.4f}'.format(sum(selectivity_arr) / len(test_sift)))

### $Z^M$ multiprobe

$$
score(\pi_t) = \Sigma_{i=1}^M\Delta_i^2(\delta_{t,i}) \\
\Delta_i(\delta) = \delta \cdot[h_i+\frac{1}{2}(1 + \delta)-\frac{a_i\cdot v + b_i}{W}]
$$

In [ ]:
def get_probe_order(q, a, b, W, M):
    real_h = (a.dot(q) + b) / W # 真实 hash 值
    h = np.floor(real_h)  # 向下取整
    positive = h - real_h + 1  # delta 取 1
    negative = -(h - real_h)  # delta 取 -1
    order = np.argsort(np.append(positive, negative)) # 排序
    sign = np.append(np.repeat(1, M), np.repeat(-1, M)) # 符号
    return order % M, sign[order]

In [ ]:
get_probe_order(q, A[0], B[0], w, M)

In [ ]:
def find_near_neighbors_multiprobe(trees, leaf_codes, q, A, B, L, W, M, T=0):
    result = []
    for i, code in enumerate(leaf_codes):
        tree = trees[i]
        node = tree[code]
        for j in range(L):
            h = np.floor((A[j].dot(q) + B[j]) / W)
            bi = generate_md5(h)
            if bi in node[j]:
                result.append(node[j][bi])
            # probe
            t = 0
            current_pos = 0
            order, sign = get_probe_order(q, A[j], B[j], W, M)
            while t < T and current_pos < len(order):
                new_h = h.copy()
                new_h[order[current_pos]] += sign[current_pos]
                bi = generate_md5(new_h)
                if bi in node[j]:
                    result.append(node[j][bi])
                    t += 1
                current_pos += 1
                
    return np.unique(np.concatenate(np.array(result)))

#### Integration

In [ ]:
L = [10]
W =  np.arange(900, 2100, 100)
M = 8
K = 50
n, d = train_sift.shape

In [ ]:
metric_results['ZM_Multiprobe'] = []

In [ ]:
%%time

for l in L:
    for w in W:
        # 1. 构造 rp 树
        forest = rp_forest_normal(P=train_sift, max_depth=4)
        # 2. 构造 hash table
        A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

        trees = construct_tables(forest, train_sift, A, B, l, w)
        
        result = dict()
        result['params'] = dict()
        result['params']['L'] = l
        result['params']['W'] = w
        
        print('L = {}, W = {}'.format(l, w))
        recall_arr = []
        error_arr = []
        selectivity_arr = []

        # 3. 查询点
        for i, q in enumerate(test_sift):
            leaf_codes = find_leaf_codes(forest, q)

            # 4. 获取 候选集 candidates multiprobe
            candidates = find_near_neighbors_multiprobe(trees, leaf_codes, q, A, B, l, w, M, T=1) 

            # 5. 计算 近似 k近邻
            approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

            # 6. 评估
            recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[i], 
                           q, train_sift)

            recall_arr.append(recall)
            error_arr.append(error)
            selectivity_arr.append(s)
            
        result['metrics'] = dict()
        result['metrics']['recall'] = sum(recall_arr) / len(test_sift)
        result['metrics']['error'] = sum(error_arr) / len(test_sift)
        result['metrics']['selectivity'] = sum(selectivity_arr) / len(test_sift)
    
        metric_results['ZM_Multiprobe'].append(result)

In [ ]:
recall_arr_ZM_Multiprobe = []
error_arr_ZM_Multiprobe = []
selectivity_arr_ZM_Multiprobe = []
for result in metric_results['ZM_Multiprobe']:
    selectivity_arr_ZM_Multiprobe.append(result['metrics']['selectivity'])
    recall_arr_ZM_Multiprobe.append(result['metrics']['recall'])
    error_arr_ZM_Multiprobe.append(result['metrics']['error'])

In [ ]:
plt.plot(selectivity_arr_ZM_Multiprobe, recall_arr_ZM_Multiprobe, marker='x', label='recall ratio')
plt.plot(selectivity_arr_ZM_Multiprobe, error_arr_ZM_Multiprobe, marker='x', label='error ratio')
plt.title('Multiprobe, $Z^M$, M=8, L=10', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
l = 10
w = 1100
# 1. 构造 rp 树
forest = rp_forest_normal(P=train_sift, max_depth=4)
# 2. 构造 hash table
A, B = generate_AB(d=d, L=l, M=M, W=w)
trees = construct_tables(forest, train_sift, A, B, l, w)

recall_arr = []
error_arr = []
selectivity_arr = []
# 3. 查询点
for i, q in enumerate(test_sift):
    leaf_codes = find_leaf_codes(forest, q)

    # 4. 获取 候选集 candidates multiprobe
    candidates = find_near_neighbors_multiprobe(trees, leaf_codes, q, A, B, l, w, M, T=1) 

    # 5. 计算 近似 k近邻
    approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

    # 6. 评估
    recall, error, s = metrics(candidates, 
                   approximate_neighbors, 
                   k_near_neighbors[i], 
                   q, train_sift)

    recall_arr.append(recall)
    error_arr.append(error)
    selectivity_arr.append(s)

print('L = {}, W = {}'.format(l, w))
print('recall = {:.4f}'.format(sum(recall_arr) / len(test_sift)))
print('error  = {:.4f}'.format(sum(error_arr) / len(test_sift)))
print('selevtivity = {:.4f}'.format(sum(selectivity_arr) / len(test_sift)))

In [ ]:
metric_results

### $E_8$ multiprobe

In [ ]:
import itertools
positions_1 = []
for i in itertools.combinations(range(8), 2):
    positions_1.append(i)

In [ ]:
signs = []
for i in itertools.product(range(-1, 2, 2), repeat=2):
    signs.append(i)

In [ ]:
type_1 = np.zeros((112, 8))

In [ ]:
i = 0
for position in positions:
    for sign in signs:
        type_1[i][position[0]] = sign[0]
        type_1[i][position[1]] = sign[1]
        i += 1

In [ ]:
type_2 = np.zeros((128, 8))

In [ ]:
type_2[:] = 0.5
type_2[1] = -0.5
type_2

In [ ]:
positions_2 = []
for i in [2, 4, 6]:
    for j in itertools.combinations(range(8), i):
        positions_2.append(j)

In [ ]:
i = 2
for position in positions_2:
    for j in position:
        type_2[i][j] *= -1
    i += 1

In [ ]:
type_2

In [ ]:
# neighbors
# type_1, type_2

In [ ]:
def find_near_neighbors_E8_multiprobe(trees, leaf_codes, q, A, B, L, W, M, T=0):
    result = []
    for i, code in enumerate(leaf_codes):
        tree = trees[i]
        node = tree[code]
        for j in range(L):
            E8 = node[j] # morton curve
            h = (A[j].dot(q) + B[j]) / W
            E8_code = E8_decode(h)
            
            bi = generate_md5(E8_code)
            if bi in E8:
                result.append(E8[bi])
            
            # 112 neighbor
            neighbor_1 = type_1 + E8_code
            distance_1 = np.array([dis.euclidean(h, neighbor) for neighbor in neighbor_1])
            
            # 128 neighbor
            neighbor_2 = type_2 + E8_code
            distance_2 = np.array([dis.euclidean(h, neighbor) for neighbor in neighbor_2])
            
            distance = np.append(distance_1, distance_2)
            order = distance.argsort()
            
            t = 0
            i = 0
            while t < T and i < 240:
                if order[i] >= 112:
                    probe_h = neighbor_2[order[i] - 112]
                else:
                    probe_h = neighbor_1[order[i]]
                bi = generate_md5(probe_h)
                if bi in E8:
                    result.append(E8[bi])
                    t += 1
                i += 1
            
    return np.unique(np.concatenate(np.array(result)))

#### Integration

In [ ]:
L = [10]
W =  np.arange(800, 2100, 100)
M = 8
K = 50
n, d = train_sift.shape

In [ ]:
metric_results['E8_Multiprobe'] = []

In [ ]:
%%time

for l in L:
    for w in W:
        # 1. 构造 rp 树
        forest = rp_forest_normal(P=train_sift, max_depth=4)
        # 2. 构造 hash table
        A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

        trees = construct_tables_E8(forest, train_sift, A, B, l, w)
        
        result = dict()
        result['params'] = dict()
        result['params']['L'] = l
        result['params']['W'] = w
        
        print('L = {}, W = {}'.format(l, w))
        recall_arr = []
        error_arr = []
        selectivity_arr = []

        # 3. 查询点
        for i, q in enumerate(test_sift):
            leaf_codes = find_leaf_codes(forest, q)

            # 4. 获取 候选集 candidates multiprobe
            candidates = find_near_neighbors_E8_multiprobe(trees, leaf_codes, q, A, B, l, w, M, T=1) 

            # 5. 计算 近似 k近邻
            approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

            # 6. 评估
            recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[i], 
                           q, train_sift)

            recall_arr.append(recall)
            error_arr.append(error)
            selectivity_arr.append(s)
            
        result['metrics'] = dict()
        result['metrics']['recall'] = sum(recall_arr) / len(test_sift)
        result['metrics']['error'] = sum(error_arr) / len(test_sift)
        result['metrics']['selectivity'] = sum(selectivity_arr) / len(test_sift)
    
        metric_results['E8_Multiprobe'].append(result)

In [ ]:
recall_arr_E8_multiprobe = []
error_arr_E8_multiprobe = []
selectivity_arr_E8_multiprobe = []
for result in metric_results['E8_Multiprobe']:
    selectivity_arr_E8_multiprobe.append(result['metrics']['selectivity'])
    recall_arr_E8_multiprobe.append(result['metrics']['recall'])
    error_arr_E8_multiprobe.append(result['metrics']['error'])

In [ ]:
plt.plot(selectivity_arr_E8_multiprobe, recall_arr_E8_multiprobe, marker='x', label='recall ratio')
plt.plot(selectivity_arr_E8_multiprobe, error_arr_E8_multiprobe, marker='x', label='error ratio')
plt.title('Multiprobe, $E_8$, M=8, L=10', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
l = 10
w = 1100
# 1. 构造 rp 树
forest = rp_forest_normal(P=train_sift, max_depth=4)
# 2. 构造 hash table
A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

trees = construct_tables_E8(forest, train_sift, A, B, l, w)

recall_arr = []
error_arr = []
selectivity_arr = []

# 3. 查询点
for i, q in enumerate(test_sift):
    leaf_codes = find_leaf_codes(forest, q)

    # 4. 获取 候选集 candidates multiprobe
    candidates = find_near_neighbors_E8_multiprobe(trees, leaf_codes, q, A, B, l, w, M, T=1) 

    # 5. 计算 近似 k近邻
    approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

    # 6. 评估
    recall, error, s = metrics(candidates, 
                   approximate_neighbors, 
                   k_near_neighbors[i], 
                   q, train_sift)

    recall_arr.append(recall)
    error_arr.append(error)
    selectivity_arr.append(s)

print('L = {}, W = {}'.format(l, w))
print('recall = {:.4f}'.format(sum(recall_arr) / len(test_sift)))
print('error  = {:.4f}'.format(sum(error_arr) / len(test_sift)))
print('selevtivity = {:.4f}'.format(sum(selectivity_arr) / len(test_sift)))

### hierarchical $Z^M$

#### Morton Curve

In [ ]:
print('Level 2: Store points into buckets')

def construct_tables_morton_curve(forest, P, A, B, L, W):
    trees = [] # n trees
    for tree in forest:
        nodes = dict() # 16 nodes
        stack = [tree['root']]
        while len(stack) != 0:
            node = stack.pop()
            if 'idxs' not in node:
                stack.append((node['r']))
                stack.append((node['l']))
            else:
                code = node['code']
                tables = [] # L buckets
                idxs = node['idxs']

                # Level 2
                for i in range(L):
                    curves = dict() # n bucket
                    H = np.floor((P[idxs].dot(A[i].T) + B[i]) / W)
                    
                    min_number = H.min()
                    if min_number < 0:
                        addition_number = -(H.min() - 1)
                    else:
                        addition_number = 0

                    curves['addition_number'] = addition_number
                    
                    H += addition_number
                    H = H.astype(np.int)
                    max_bit = len(bin(H.max())[2:])
                    curves['max_bit'] = max_bit
                    
                    morton_number = np.array([morton_curve(h, max_bit=max_bit) for h in H])
                    
                    buckets = dict()
                    
                    for j, number in enumerate(morton_number): # morton 唯一标识
                        if number in buckets:
                            buckets[number].append(idxs[j])
                        else:
                            buckets[number] = [idxs[j]]
                    
                    morton_number.sort()
                    curves['morton_number'] = morton_number # morton 码索引列表
                    curves['buckets'] = buckets
                    
                    tables.append(curves)

                nodes[code] = tables

        trees.append(nodes)

    return trees

In [ ]:
def find_near_neighbors_morton(trees, leaf_codes, q, A, B, L, W, M, T=0):
    result = []
    for i, code in enumerate(leaf_codes):
        tree = trees[i]
        node = tree[code]
        for j in range(L):
            curve = node[j] # morton curve
            h = np.floor((A[j].dot(q) + B[j]) / W)
            h += curve['addition_number']
            h = h.astype(np.int)
            
            morton_number = morton_curve(h, curve['max_bit'])
            
            idx = np.argwhere(curve['morton_number'] < morton_number)
            if len(idx) == 0: # 最小
                idx = 0
            elif len(idx) == len(curve['morton_number']):
                idx = idx[-2][0]
            elif len(idx) == len(curve['morton_number']) - 1:
                idx = idx[-3][0]
            else:
                idx = idx[-1][0]

            m1 = curve['morton_number'][idx]            
            # print(idx)
            if morton_number in curve['buckets']:
                result.append(curve['buckets'][morton_number])
                m2 = curve['morton_number'][idx + 2]
            else:
                m2 = curve['morton_number'][idx + 1]
            
            # probe
            t = 0
            result.append(curve['buckets'][m1])
            result.append(curve['buckets'][m2])
                
    return np.unique(np.concatenate(np.array(result)))

#### Integration

In [ ]:
L = [10]
W =  np.arange(800, 2100, 100)
M = 8
K = 50
n, d = train_sift.shape

In [ ]:
metric_results['ZM_hierarchical'] = []

In [ ]:
%%time

for l in L:
    for w in W:
        # 1. 构造 rp 树
        forest = rp_forest_normal(P=train_sift, max_depth=4)
        # 2. 构造 hash table
        A, B = generate_AB(d=d, L=l, M=M, W=w, random_state=1)

        trees = construct_tables_morton_curve(forest, train_sift, A, B, l, w)
        
        result = dict()
        result['params'] = dict()
        result['params']['L'] = l
        result['params']['W'] = w
        
        print('L = {}, W = {}'.format(l, w))
        recall_arr = []
        error_arr = []
        selectivity_arr = []

        # 3. 查询点
        for i, q in enumerate(test_sift):
            leaf_codes = find_leaf_codes(forest, q)

            # 4. 获取 候选集 candidates multiprobe
            candidates = find_near_neighbors_morton(trees, leaf_codes, q, A, B, l, w, M, T=1) 

            # 5. 计算 近似 k近邻
            approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

            # 6. 评估
            recall, error, s = metrics(candidates, 
                           approximate_neighbors, 
                           k_near_neighbors[i], 
                           q, train_sift)

            recall_arr.append(recall)
            error_arr.append(error)
            selectivity_arr.append(s)

        result['metrics'] = dict()
        result['metrics']['recall'] = sum(recall_arr) / len(test_sift)
        result['metrics']['error'] = sum(error_arr) / len(test_sift)
        result['metrics']['selectivity'] = sum(selectivity_arr) / len(test_sift)
    
        metric_results['ZM_hierarchical'].append(result)

In [ ]:
recall_arr_ZM_hierarchical = []
error_arr_ZM_hierarchical = []
selectivity_arr_ZM_hierarchical = []
for result in metric_results['ZM_hierarchical']:
    selectivity_arr_ZM_hierarchical.append(result['metrics']['selectivity'])
    recall_arr_ZM_hierarchical.append(result['metrics']['recall'])
    error_arr_ZM_hierarchical.append(result['metrics']['error'])

In [ ]:
plt.plot(selectivity_arr_ZM_hierarchical, recall_arr_ZM_hierarchical, marker='x', label='recall ratio')
plt.plot(selectivity_arr_ZM_hierarchical, error_arr_ZM_hierarchical, marker='x', label='error ratio')
plt.title('Multiprobe, $E_8$, M=8, L=10', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
l = 10
w = 1100
# 1. 构造 rp 树
forest = rp_forest_normal(P=train_sift, max_depth=4)
# 2. 构造 hash table
A, B = generate_AB(d=d, L=l, M=M, W=w)
trees = construct_tables_morton_curve(forest, train_sift, A, B, l, w)

recall_arr = []
error_arr = []
selectivity_arr = []
# 3. 查询点
for i, q in enumerate(test_sift):
    leaf_codes = find_leaf_codes(forest, q)

    # 4. 获取 候选集 candidates multiprobe
    candidates = find_near_neighbors_morton(trees, leaf_codes, q, A, B, l, w, M, T=1) 

    # 5. 计算 近似 k近邻
    approximate_neighbors = find_k_nearest_neighbors(train_sift, q, candidates)

    # 6. 评估
    recall, error, s = metrics(candidates, 
                   approximate_neighbors, 
                   k_near_neighbors[i], 
                   q, train_sift)

    recall_arr.append(recall)
    error_arr.append(error)
    selectivity_arr.append(s)

print('L = {}, W = {}'.format(l, w))
print('recall = {:.4f}'.format(sum(recall_arr) / len(test_sift)))
print('error  = {:.4f}'.format(sum(error_arr) / len(test_sift)))
print('selevtivity = {:.4f}'.format(sum(selectivity_arr) / len(test_sift)))

### hierarchical $E_8$

In [ ]:
def hierarchical_E8(E8_codes):
    E8_codes_unique = np.unique(E8_codes, axis=0)
    
    temp = E8_codes_unique.copy()
    m = 0
    temp_levels = []
    while len(temp) != 1:
        m += 1
        temp /= 2
        temp = np.array([E8_decode(code) for code in temp])
        temp = np.unique(temp, axis=0)
        
        current_temp = temp * (2 ** m) # current level HE8 code
        temp_levels.append(np.array([generate_md5(code) for code in current_temp]))

    print(m)

    temp_levels = np.array(temp_levels)

    levels = dict()
    node = dict()
    node['code'] = temp_levels[-1][0]
    levels['root'] = node
    
    

In [ ]:
print('Level 2: Store points into buckets')

def construct_tables_E8_hierarchical(forest, P, A, B, L, W):
    trees = [] # n trees
    for tree in forest:
        nodes = dict() # 16 nodes
        stack = [tree['root']]
        while len(stack) != 0:
            node = stack.pop()
            if 'idxs' not in node:
                stack.append((node['r']))
                stack.append((node['l']))
            else:
                code = node['code']
                tables = [] # L buckets
                idxs = node['idxs']

                # Level 2
                for i in range(L):
                    E8 = dict() # n bucket
                    E8_codes = []
                    H = (P[idxs].dot(A[i].T) + B[i]) / W
                    
                    for j, h in enumerate(H):
                        E8_code = E8_decode(h)
                        E8_codes.append(E8_code)
                        bi = generate_md5(E8_code)
                        if bi in E8:
                            E8[bi].append(idxs[j])
                        else:
                            E8[bi] = [idxs[j]]
                    level_E8 = hierarchical_E8(E8_codes)
                    tables.append(E8)

                nodes[code] = tables

        trees.append(nodes)

    return trees

### Integration

In [ ]:
len(recall_arr)

In [ ]:
x = np.linspace(0.02, 0.8, 13)

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8, recall_arr_E8, marker='x', label='$E_8$ Standard')
plt.plot(selectivity_arr_E8_multiprobe, recall_arr_E8_multiprobe, marker='x', label='$E_8$ Multiprobe')
plt.plot(selectivity_arr_ZM_Multiprobe, recall_arr_ZM_Multiprobe, marker='x', label='$Z^M$ Multiprobe')
plt.plot(selectivity_arr_ZM_hierarchical, recall_arr_ZM_hierarchical, marker='x', label='$Z^M$ Hierarchical')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, error_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8, error_arr_E8, marker='x', label='$E_8$ Standard')
plt.plot(selectivity_arr_E8_multiprobe, error_arr_E8_multiprobe, marker='x', label='$E_8$ Multiprobe')
plt.plot(selectivity_arr_ZM_Multiprobe, error_arr_ZM_Multiprobe, marker='x', label='$Z^M$ Multiprobe')
plt.plot(selectivity_arr_ZM_hierarchical, error_arr_ZM_hierarchical, marker='x', label='$Z^M$ Hierarchical')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8, recall_arr_E8, marker='x', label='$E_8$ Standard')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8_multiprobe, recall_arr_E8_multiprobe, marker='x', label='$E_8$ Multiprobe')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_ZM_Multiprobe, recall_arr_ZM_Multiprobe, marker='x', label='$Z^M$ Multiprobe')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, recall_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_ZM_hierarchical, recall_arr_ZM_hierarchical, marker='x', label='$Z^M$ Hierarchical')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, error_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8, error_arr_E8, marker='x', label='$E_8$ Standard')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, error_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_E8_multiprobe, error_arr_E8_multiprobe, marker='x', label='$E_8$ Multiprobe')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, error_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_ZM_Multiprobe, error_arr_ZM_Multiprobe, marker='x', label='$Z^M$ Multiprobe')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.plot(selectivity_arr, error_arr, marker='x', label='$Z^M$ Standard')
plt.plot(selectivity_arr_ZM_hierarchical, error_arr_ZM_hierarchical, marker='x', label='$Z^M$ Hierarchical')
plt.title('M=8, L=10, K=50', fontsize=12)
plt.xlabel('selectivity', fontsize=12)
plt.ylabel('quality', fontsize=12)
plt.legend(fontsize=12)
plt.show()